In [1]:
import pandas as pd
import numpy as np
import os
import random
from copy import deepcopy
import math
from itertools import product
import pickle
import joblib

import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
user_df = pd.read_csv('./data/user_spec.csv')
loan_df = pd.read_csv('./data/loan_result.csv')
# log_df = pd.read_csv('./data/log_data.csv' )

In [3]:
user_df.shape, loan_df.shape

((1394216, 17), (13527363, 7))

In [4]:
loan_df = loan_df.loc[loan_df['is_applied'].dropna(axis=0).index]

In [5]:
merged_df=pd.merge(user_df, loan_df, on='application_id')

In [6]:
merged_df.shape

(10270011, 23)

In [7]:
merged_df.columns

Index(['application_id', 'user_id', 'birth_year', 'gender', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'existing_loan_amt', 'loanapply_insert_time',
       'bank_id', 'product_id', 'loan_limit', 'loan_rate', 'is_applied'],
      dtype='object')

In [8]:
merged_df.isnull().sum()

application_id                               0
user_id                                      0
birth_year                               91626
gender                                   91626
insert_time                                  0
credit_score                           1243812
yearly_income                                0
income_type                                  0
company_enter_month                     303568
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn             5873229
personal_rehabilitation_complete_yn    9232232
existing_loan_cnt                      2143811
existing_loan_amt                      3044140
loanapply_insert_time                        0
bank_id                                      0
product_id                                   0
loan_limit                                5625
loan_rate    

In [9]:
merged_df['age'] = 2022 - merged_df['birth_year']
merged_df.drop([
    'application_id','user_id','birth_year','insert_time',
    'loanapply_insert_time', 'bank_id','product_id',
    'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
    ]
    , axis=1, inplace=True)

## 기본형 분석 + drop

In [10]:
data_set=merged_df.copy()

In [11]:
data_set.drop([
    'income_type',
    'employment_type',
    'houseown_type',
    'purpose',
]
    , axis=1, inplace=True)

In [12]:
data_set.dropna(inplace=True)

In [13]:
data_set.shape

(6718484, 11)

In [14]:
data_set = pd.concat([data_set[data_set['is_applied']==0].sample(100_000, random_state=42), data_set[data_set['is_applied']==1].sample(100_000, random_state=42)])

In [15]:
X,y= data_set.loc[:, data_set.columns !='is_applied'] ,data_set['is_applied']
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=42)

In [16]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [17]:
model.score(X_test, y_test)

0.7777

In [18]:
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.7849784785026843

In [19]:
confusion_matrix(y_test, y_pred)

array([[14877,  5116],
       [ 3776, 16231]])

In [20]:
print(*sorted(zip(model.feature_importances_, X_test.columns), reverse=True), sep='\n')

(0.19228858270096455, 'credit_score')
(0.16373669266424057, 'loan_rate')
(0.10545242117031779, 'loan_limit')
(0.10043875308038004, 'desired_amount')
(0.09979546862152754, 'existing_loan_amt')
(0.09312046012778558, 'company_enter_month')
(0.08820785104862722, 'yearly_income')
(0.0859836324978241, 'age')
(0.05713868792678408, 'existing_loan_cnt')
(0.013837450161548548, 'gender')


## get_dummy + drop

In [46]:
data_set=merged_df.copy()

In [47]:
data_set.dropna(inplace=True)

In [48]:
data_set = pd.concat([data_set[data_set['is_applied']==0].sample(100_000, random_state=42), data_set[data_set['is_applied']==1].sample(100_000, random_state=42)])

In [49]:
data_set.shape

(200000, 15)

In [10]:
cate_list = [
    'income_type',
    'employment_type',
    'houseown_type',
    'purpose',
#     'personal_rehabilitation_yn',
#     'personal_rehabilitation_complete_yn',
]

In [51]:
for cate in cate_list:
    data_set = pd.get_dummies(data = data_set, columns=[cate], prefix=cate)

In [52]:
data_set.columns

Index(['gender', 'credit_score', 'yearly_income', 'company_enter_month',
       'desired_amount', 'existing_loan_cnt', 'existing_loan_amt',
       'loan_limit', 'loan_rate', 'is_applied', 'age',
       'income_type_EARNEDINCOME', 'income_type_EARNEDINCOME2',
       'income_type_FREELANCER', 'income_type_OTHERINCOME',
       'income_type_PRACTITIONER', 'income_type_PRIVATEBUSINESS',
       'employment_type_계약직', 'employment_type_기타', 'employment_type_일용직',
       'employment_type_정규직', 'houseown_type_기타가족소유', 'houseown_type_배우자',
       'houseown_type_자가', 'houseown_type_전월세', 'purpose_BUSINESS',
       'purpose_BUYCAR', 'purpose_BUYHOUSE', 'purpose_ETC',
       'purpose_HOUSEDEPOSIT', 'purpose_INVEST', 'purpose_LIVING',
       'purpose_SWITCHLOAN', 'purpose_기타', 'purpose_대환대출', 'purpose_사업자금',
       'purpose_생활비', 'purpose_자동차구입', 'purpose_전월세보증금', 'purpose_주택구입',
       'purpose_투자'],
      dtype='object')

In [53]:
X,y= data_set.loc[:, data_set.columns !='is_applied'] ,data_set['is_applied']
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=42)

In [54]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [55]:
model.score(X_test, y_test)

0.787425

In [56]:
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.7940214626583658

In [57]:
confusion_matrix(y_test, y_pred)

array([[15108,  4885],
       [ 3618, 16389]])

In [58]:
print(*sorted(zip(model.feature_importances_, X_test.columns), reverse=True), sep='\n')

(0.17717951451829048, 'credit_score')
(0.1536787724427905, 'loan_rate')
(0.09940527040932981, 'loan_limit')
(0.09150185835829833, 'desired_amount')
(0.08173739386260723, 'existing_loan_amt')
(0.07890960552409164, 'company_enter_month')
(0.07288739553121247, 'yearly_income')
(0.07248563232373308, 'age')
(0.05698642301275427, 'existing_loan_cnt')
(0.016614355908840542, 'income_type_EARNEDINCOME')
(0.011803254927539617, 'gender')
(0.010866493381125261, 'purpose_생활비')
(0.008828559093822418, 'houseown_type_전월세')
(0.008432704437602802, 'houseown_type_기타가족소유')
(0.007702812564903346, 'houseown_type_자가')
(0.007003873207271149, 'purpose_대환대출')
(0.006642881257334032, 'employment_type_정규직')
(0.0056735095298231985, 'employment_type_기타')
(0.004990439672428415, 'income_type_PRIVATEBUSINESS')
(0.004801543741054417, 'employment_type_계약직')
(0.0038961058688042477, 'income_type_EARNEDINCOME2')
(0.002470783405348363, 'purpose_사업자금')
(0.0023778252501610564, 'purpose_주택구입')
(0.002243420745427083, 'income_typ

## get_dummies + remove_duplicate + drop

In [11]:
for cate in cate_list:
    merged_df = pd.get_dummies(data = merged_df, columns=[cate], prefix=cate)

In [12]:
[i for i in merged_df.columns if 'purpose' in i]

['purpose_BUSINESS',
 'purpose_BUYCAR',
 'purpose_BUYHOUSE',
 'purpose_ETC',
 'purpose_HOUSEDEPOSIT',
 'purpose_INVEST',
 'purpose_LIVING',
 'purpose_SWITCHLOAN',
 'purpose_기타',
 'purpose_대환대출',
 'purpose_사업자금',
 'purpose_생활비',
 'purpose_자동차구입',
 'purpose_전월세보증금',
 'purpose_주택구입',
 'purpose_투자']

In [12]:
merged_df[ 'purpose_주택구입'] += merged_df[ 'purpose_BUYHOUSE']
merged_df[ 'purpose_자동차구입'] += merged_df[ 'purpose_BUYCAR']
merged_df[ 'purpose_사업자금'] += merged_df[ 'purpose_BUSINESS']
merged_df[ 'purpose_기타'] += merged_df[ 'purpose_ETC']
merged_df[ 'purpose_전월세보증금'] += merged_df[ 'purpose_HOUSEDEPOSIT']
merged_df[ 'purpose_투자'] += merged_df[ 'purpose_INVEST']
merged_df[ 'purpose_생활비'] += merged_df[ 'purpose_LIVING']
merged_df[ 'purpose_대환대출'] += merged_df[ 'purpose_SWITCHLOAN']
merged_df.drop(['purpose_BUYHOUSE','purpose_BUYCAR','purpose_BUSINESS','purpose_ETC','purpose_HOUSEDEPOSIT',
               'purpose_INVEST','purpose_LIVING','purpose_SWITCHLOAN'], axis=1, inplace=True)

In [15]:
data_set=merged_df.copy()

In [16]:
data_set.dropna(inplace=True)

In [17]:
data_set = pd.concat([data_set[data_set['is_applied']==0].sample(100_000, random_state=42), data_set[data_set['is_applied']==1].sample(100_000, random_state=42)])

In [18]:
data_set.shape

(200000, 33)

In [19]:
data_set.columns

Index(['gender', 'credit_score', 'yearly_income', 'company_enter_month',
       'desired_amount', 'existing_loan_cnt', 'existing_loan_amt',
       'loan_limit', 'loan_rate', 'is_applied', 'age',
       'income_type_EARNEDINCOME', 'income_type_EARNEDINCOME2',
       'income_type_FREELANCER', 'income_type_OTHERINCOME',
       'income_type_PRACTITIONER', 'income_type_PRIVATEBUSINESS',
       'employment_type_계약직', 'employment_type_기타', 'employment_type_일용직',
       'employment_type_정규직', 'houseown_type_기타가족소유', 'houseown_type_배우자',
       'houseown_type_자가', 'houseown_type_전월세', 'purpose_기타', 'purpose_대환대출',
       'purpose_사업자금', 'purpose_생활비', 'purpose_자동차구입', 'purpose_전월세보증금',
       'purpose_주택구입', 'purpose_투자'],
      dtype='object')

In [20]:
X,y= data_set.loc[:, data_set.columns !='is_applied'] ,data_set['is_applied']
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=42)

In [21]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [22]:
model.score(X_test, y_test)

0.78925

In [23]:
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.7959331880900509

In [24]:
confusion_matrix(y_test, y_pred)

array([[15130,  4863],
       [ 3567, 16440]])

In [25]:
print(*sorted(zip(model.feature_importances_, X_test.columns), reverse=True), sep='\n')

(0.17840107245459003, 'credit_score')
(0.1530654975230667, 'loan_rate')
(0.09873250904273409, 'loan_limit')
(0.08911375637957228, 'desired_amount')
(0.08145241822926808, 'existing_loan_amt')
(0.07917246715567108, 'company_enter_month')
(0.07316431410498936, 'age')
(0.07297254060600435, 'yearly_income')
(0.0574061379256571, 'existing_loan_cnt')
(0.017014337764825374, 'income_type_EARNEDINCOME')
(0.01219894977686387, 'gender')
(0.011140343328215379, 'purpose_생활비')
(0.009176220776415412, 'houseown_type_전월세')
(0.00867786753832694, 'houseown_type_기타가족소유')
(0.007808819126192054, 'houseown_type_자가')
(0.007566391772735274, 'purpose_대환대출')
(0.00702038104674273, 'employment_type_정규직')
(0.0053873570762820455, 'employment_type_기타')
(0.005043456602135085, 'income_type_PRIVATEBUSINESS')
(0.0049492906454045235, 'employment_type_계약직')
(0.004043020177306242, 'income_type_EARNEDINCOME2')
(0.002524264520563109, 'income_type_FREELANCER')
(0.0025106787942271638, 'purpose_사업자금')
(0.0021847375239532605, 'pur

# 돌려보기

편의를 위해서 gender과 age null은 drop

In [13]:
merged_df.columns

Index(['gender', 'credit_score', 'yearly_income', 'company_enter_month',
       'desired_amount', 'existing_loan_cnt', 'existing_loan_amt',
       'loan_limit', 'loan_rate', 'is_applied', 'age',
       'income_type_EARNEDINCOME', 'income_type_EARNEDINCOME2',
       'income_type_FREELANCER', 'income_type_OTHERINCOME',
       'income_type_PRACTITIONER', 'income_type_PRIVATEBUSINESS',
       'employment_type_계약직', 'employment_type_기타', 'employment_type_일용직',
       'employment_type_정규직', 'houseown_type_기타가족소유', 'houseown_type_배우자',
       'houseown_type_자가', 'houseown_type_전월세', 'purpose_기타', 'purpose_대환대출',
       'purpose_사업자금', 'purpose_생활비', 'purpose_자동차구입', 'purpose_전월세보증금',
       'purpose_주택구입', 'purpose_투자'],
      dtype='object')

In [14]:
merged_df = merged_df.loc[merged_df['gender'].dropna(axis=0).index]

In [15]:
merged_df.isnull().sum()[merged_df.isnull().sum()!=0]

credit_score           1229941
company_enter_month     299731
existing_loan_cnt      2101116
existing_loan_amt      2994214
loan_limit                5584
loan_rate                 5584
dtype: int64

In [16]:
# null_feature = merged_df.isnull().sum()[merged_df.isnull().sum()!=0].index
null_feature = ['existing_loan_cnt', 'company_enter_month', 'existing_loan_amt',
       'loan_limit', 'loan_rate', 'credit_score']
options = [0, 'min', 'max', 'median']
random_list = [random.randint(1,1000) for i in range(len(options)**len(null_feature))]
len(random_list)

4096

In [17]:
# model_list = []
acc_list = []
f1_list = []
confusion_list = []
my_options=list(product(options, repeat=len(null_feature)))
for i, option_group in enumerate(my_options):
    tmp = deepcopy(merged_df)
    data_set = pd.concat([tmp[tmp['is_applied']==0].sample(100_000, random_state=random_list[i]), tmp[tmp['is_applied']==1].sample(100_000, random_state=random_list[i])])
    
    for column_name, option in zip(null_feature, option_group):
        if option == 0:
            data_set[column_name].fillna(0, inplace=True)
        else:
            eval(f'data_set["{column_name}"].fillna(data_set["{column_name}"].{option}(), inplace=True)')

    X,y= data_set.loc[:, data_set.columns !='is_applied'] ,data_set['is_applied']
    X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=random_list[i])
    model = RandomForestClassifier(random_state=random_list[i])
    model.fit(X_train, y_train)
    y_pred=model.predict(X_test)
    
#     model_list.append(model)
    acc_list.append(model.score(X_test,y_test))
    f1_list.append(f1_score(y_test,y_pred))
    confusion_list.append(confusion_matrix(y_test,y_pred))
    
#     with open(f'./mydata/m_{option_group}.txt','w') as f:
#         f.write('\n'.join(map(str,data_set.index)))
#     saved_model = pickle.dumps(model)
#     joblib.dump(model,f'./model/m_m_{option_group}.pkl')
    print(acc_list[-1], f1_list[-1], "/// index:",i, option_group, 'done', )

0.785 0.7921500386697603 /// index: 0 (0, 0, 0, 0, 0, 0) done
0.791625 0.797019214377907 /// index: 1 (0, 0, 0, 0, 0, 'min') done
0.7851 0.7919147906076011 /// index: 2 (0, 0, 0, 0, 0, 'max') done
0.79085 0.7954923242397575 /// index: 3 (0, 0, 0, 0, 0, 'median') done
0.786925 0.7919443427316002 /// index: 4 (0, 0, 0, 0, 'min', 0) done
0.78795 0.793725680933852 /// index: 5 (0, 0, 0, 0, 'min', 'min') done
0.784575 0.7901619383903568 /// index: 6 (0, 0, 0, 0, 'min', 'max') done
0.78075 0.7863476905086726 /// index: 7 (0, 0, 0, 0, 'min', 'median') done
0.785575 0.7918153353236729 /// index: 8 (0, 0, 0, 0, 'max', 0) done
0.784 0.7912337505436621 /// index: 9 (0, 0, 0, 0, 'max', 'min') done
0.784425 0.7906784803981064 /// index: 10 (0, 0, 0, 0, 'max', 'max') done
0.78575 0.7902183491628317 /// index: 11 (0, 0, 0, 0, 'max', 'median') done
0.78325 0.7907818532818534 /// index: 12 (0, 0, 0, 0, 'median', 0) done
0.783 0.7893510653788284 /// index: 13 (0, 0, 0, 0, 'median', 'min') done
0.787575 

0.787325 0.7923653315759929 /// index: 107 (0, 0, 'min', 'max', 'max', 'median') done
0.787875 0.795152217474228 /// index: 108 (0, 0, 'min', 'max', 'median', 0) done
0.787475 0.7941197839722943 /// index: 109 (0, 0, 'min', 'max', 'median', 'min') done
0.78685 0.7931185091720857 /// index: 110 (0, 0, 'min', 'max', 'median', 'max') done
0.786075 0.7927434785767916 /// index: 111 (0, 0, 'min', 'max', 'median', 'median') done
0.78645 0.793252008906961 /// index: 112 (0, 0, 'min', 'median', 0, 0) done
0.7875 0.7939294026377036 /// index: 113 (0, 0, 'min', 'median', 0, 'min') done
0.786475 0.792356502078623 /// index: 114 (0, 0, 'min', 'median', 0, 'max') done
0.7874 0.7933715618621829 /// index: 115 (0, 0, 'min', 'median', 0, 'median') done
0.786625 0.7933364003971041 /// index: 116 (0, 0, 'min', 'median', 'min', 0) done
0.7854 0.7923661167819651 /// index: 117 (0, 0, 'min', 'median', 'min', 'min') done
0.785225 0.7913945074423911 /// index: 118 (0, 0, 'min', 'median', 'min', 'max') done
0

0.782775 0.789270729755293 /// index: 208 (0, 0, 'median', 'min', 0, 0) done
0.789825 0.7951560634487465 /// index: 209 (0, 0, 'median', 'min', 0, 'min') done
0.7832 0.78914608052908 /// index: 210 (0, 0, 'median', 'min', 0, 'max') done
0.78395 0.7909024921364627 /// index: 211 (0, 0, 'median', 'min', 0, 'median') done
0.788175 0.7951451850777302 /// index: 212 (0, 0, 'median', 'min', 'min', 0) done
0.78725 0.7945932898865555 /// index: 213 (0, 0, 'median', 'min', 'min', 'min') done
0.784575 0.7906106480693994 /// index: 214 (0, 0, 'median', 'min', 'min', 'max') done
0.7828 0.7898098417767456 /// index: 215 (0, 0, 'median', 'min', 'min', 'median') done
0.785275 0.7914835765094316 /// index: 216 (0, 0, 'median', 'min', 'max', 0) done
0.791125 0.7985339152660896 /// index: 217 (0, 0, 'median', 'min', 'max', 'min') done
0.78665 0.7913651476628202 /// index: 218 (0, 0, 'median', 'min', 'max', 'max') done
0.789 0.7939654330631774 /// index: 219 (0, 0, 'median', 'min', 'max', 'median') done


0.781575 0.7881990739618434 /// index: 307 (0, 'min', 0, 'median', 0, 'median') done
0.7849 0.7915697674418604 /// index: 308 (0, 'min', 0, 'median', 'min', 0) done
0.782825 0.7892067651840529 /// index: 309 (0, 'min', 0, 'median', 'min', 'min') done
0.7839 0.7897246278096721 /// index: 310 (0, 'min', 0, 'median', 'min', 'max') done
0.78785 0.7938089221498689 /// index: 311 (0, 'min', 0, 'median', 'min', 'median') done
0.7839 0.7910260129581279 /// index: 312 (0, 'min', 0, 'median', 'max', 0) done
0.78225 0.7887665518746665 /// index: 313 (0, 'min', 0, 'median', 'max', 'min') done
0.78585 0.7917537803277095 /// index: 314 (0, 'min', 0, 'median', 'max', 'max') done
0.786875 0.7926749191371386 /// index: 315 (0, 'min', 0, 'median', 'max', 'median') done
0.78465 0.7925537038820923 /// index: 316 (0, 'min', 0, 'median', 'median', 0) done
0.7853 0.7920983828798295 /// index: 317 (0, 'min', 0, 'median', 'median', 'min') done
0.78265 0.7887036406941136 /// index: 318 (0, 'min', 0, 'median', '

0.783975 0.790078468527561 /// index: 404 (0, 'min', 'max', 'min', 'min', 0) done
0.782675 0.7897244865871652 /// index: 405 (0, 'min', 'max', 'min', 'min', 'min') done
0.784625 0.7898114036157807 /// index: 406 (0, 'min', 'max', 'min', 'min', 'max') done
0.787925 0.7947545425951464 /// index: 407 (0, 'min', 'max', 'min', 'min', 'median') done
0.785925 0.7929792326475352 /// index: 408 (0, 'min', 'max', 'min', 'max', 0) done
0.784225 0.7902806463370184 /// index: 409 (0, 'min', 'max', 'min', 'max', 'min') done
0.782825 0.7881064468131815 /// index: 410 (0, 'min', 'max', 'min', 'max', 'max') done
0.7874 0.7937824336776759 /// index: 411 (0, 'min', 'max', 'min', 'max', 'median') done
0.78025 0.7866401281615613 /// index: 412 (0, 'min', 'max', 'min', 'median', 0) done
0.7867 0.7929126213592232 /// index: 413 (0, 'min', 'max', 'min', 'median', 'min') done
0.78415 0.7886827549072397 /// index: 414 (0, 'min', 'max', 'min', 'median', 'max') done
0.7856 0.7915714771788266 /// index: 415 (0, 'm

0.7824 0.790204396451986 /// index: 499 (0, 'min', 'median', 'median', 0, 'median') done
0.786075 0.7925927721356375 /// index: 500 (0, 'min', 'median', 'median', 'min', 0) done
0.7866 0.7928255909907286 /// index: 501 (0, 'min', 'median', 'median', 'min', 'min') done
0.78615 0.792136469673406 /// index: 502 (0, 'min', 'median', 'median', 'min', 'max') done
0.7862 0.791047693510555 /// index: 503 (0, 'min', 'median', 'median', 'min', 'median') done
0.786125 0.7924601538050994 /// index: 504 (0, 'min', 'median', 'median', 'max', 0) done
0.784275 0.7903394319313848 /// index: 505 (0, 'min', 'median', 'median', 'max', 'min') done
0.7859 0.7893856672077124 /// index: 506 (0, 'min', 'median', 'median', 'max', 'max') done
0.7839 0.7900412922030605 /// index: 507 (0, 'min', 'median', 'median', 'max', 'median') done
0.782775 0.7875394283199257 /// index: 508 (0, 'min', 'median', 'median', 'median', 0) done
0.785675 0.792627173991921 /// index: 509 (0, 'min', 'median', 'median', 'median', 'min'

0.784175 0.7895262939756686 /// index: 598 (0, 'max', 'min', 'min', 'min', 'max') done
0.789375 0.7948274602440153 /// index: 599 (0, 'max', 'min', 'min', 'min', 'median') done
0.78475 0.7906026557711949 /// index: 600 (0, 'max', 'min', 'min', 'max', 0) done
0.782925 0.7892526880415525 /// index: 601 (0, 'max', 'min', 'min', 'max', 'min') done
0.788925 0.7945592135679003 /// index: 602 (0, 'max', 'min', 'min', 'max', 'max') done
0.784375 0.7906197654941373 /// index: 603 (0, 'max', 'min', 'min', 'max', 'median') done
0.78255 0.7880604288499026 /// index: 604 (0, 'max', 'min', 'min', 'median', 0) done
0.785075 0.7902914989632882 /// index: 605 (0, 'max', 'min', 'min', 'median', 'min') done
0.7815 0.7881930981000388 /// index: 606 (0, 'max', 'min', 'min', 'median', 'max') done
0.786 0.7927160015497869 /// index: 607 (0, 'max', 'min', 'min', 'median', 'median') done
0.7862 0.7925681575628215 /// index: 608 (0, 'max', 'min', 'max', 0, 0) done
0.786625 0.7926334459049054 /// index: 609 (0, 

KeyboardInterrupt: 

In [ ]:
print(*sorted(list(zip(acc_list,f1_list, my_options)[:30]))), sep='\n')